In [1]:
import pandas as pd
import re
import json

In [2]:
def process_pairs(row_data, poker_odds_dict):

    pair_reg = re.compile(r'Pair of [0-9AKQJT]\'s')
    if pair_reg.search(row_data.Cards) is not None:
        # win rate:     
        win_data = float(row_data.Win[:-1])
        
        # dictionary key:
        pair_key = row_data.Cards[-3]        
        
        # add dictionary :
        poker_odds_dict[pair_key] = win_data
        
    return poker_odds_dict

In [3]:
def process_suited_cards(row_data, poker_odds_dict):

    suited_reg = re.compile(r'[0-9AKQJT]/[0-9AKQJT] suited')
    if suited_reg.search(row_data.Cards) is not None:
        # win rate:     
        win_data = float(row_data.Win[:-1])
        
        # dictionary key:
        first_card, second_card = row_data.Cards.split("/")
        second_card = second_card[0]
        first_card, second_card = swap_cards(first_card, second_card)    
        card_key = first_card + second_card
        
        # add dictionary
        poker_odds_dict[card_key] = {'s': win_data, "o": 0}
        
    return poker_odds_dict

In [4]:
def process_unsuited_cards(row_data, poker_odds_dict):
    if "unsuited" in row_data.Cards:
        # win rate:     
        win_data = float(row_data.Win[:-1])
        
        # dictionary key:
        first_card, second_card = row_data.Cards.split("/")
        second_card = second_card[0]
        first_card, second_card = swap_cards(first_card, second_card)  
        card_key = first_card + second_card
        
        # update key (process_suided_cards() must be ran before process_unsuited_cards)
        poker_odds_dict[card_key]['o'] = win_data
        
    return poker_odds_dict

In [5]:
def swap_cards(first_card, second_card):
    card_values = {
        "2": 2,
        "3": 3,
        "4": 4,
        "5": 5,
        "6": 6,
        "7": 7,
        "8": 8,
        "9": 9,
        "T": 10,
        "J": 11,
        "Q": 12,
        "K": 13,
        "A": 14
    }
    
    if card_values[first_card] > card_values[second_card]:
        return first_card, second_card
    else:
        return second_card, first_card

In [6]:
def build_json_files(players):
    # raw data from wizardofodds.com
    wizardofodds_url = "https://wizardofodds.com/games/texas-hold-em/{}-player-game/".format(players)
    poker_odds_raw= pd.read_html(wizardofodds_url)


    
    # # 2 player odds:
    # poker_odds_raw= pd.read_html("https://wizardofodds.com/games/texas-hold-em/2-player-game/")
    # filename = "2_player_odds.json"

    # # 3 player odds:
    # poker_odds_raw= pd.read_html("https://wizardofodds.com/games/texas-hold-em/3-player-game/")
    # filename = "3_player_odds.json"

    # # 4 player odds:
    # poker_odds_raw= pd.read_html("https://wizardofodds.com/games/texas-hold-em/4-player-game/")
    # filename = "4_player_odds.json"

    # Clean up:
    poker_odds = poker_odds_raw[0]
    poker_odds.columns = poker_odds.iloc[0, :]
    poker_odds = poker_odds.iloc[1:,:]
    poker_odds = poker_odds.iloc[:, 0:2]
    poker_odds.columns = ["Cards", "Win"]

    # Build the odds dictionary from the dataframe:
    poker_odds_dict = {}
    # process pairs:
    poker_odds_dict = poker_odds.apply(process_pairs, axis=1, args=(poker_odds_dict, ))
    poker_odds_dict = poker_odds_dict.iloc[-1]
    # process suited cards:
    poker_odds_dict = poker_odds.apply(process_suited_cards, axis=1, args=(poker_odds_dict, ))
    poker_odds_dict = poker_odds_dict.iloc[-1]
    # process unsuited cards:
    poker_odds_dict = poker_odds.apply(process_unsuited_cards, axis=1, args=(poker_odds_dict, ))
    poker_odds_dict = poker_odds_dict.iloc[-1]

    # convert the odds dictionary to json and write to a file
    filename = "{}_player_odds.json".format(players)
    json_file = open(filename, 'w')
    json_file.write(json.dumps(poker_odds_dict))
    json_file.close()

In [7]:
players_list = [2, 3, 4, 6, 8, 10]

for players in players_list:
    build_json_files(players)